In [1]:
import pandas as pd
import numpy as np


# 读取 Excel 文件
df = pd.read_excel('field_activity_M.xlsx')


# 保留所需字段和 2023-12 至 2024-11 的所有字段
columns_to_keep = ['id', 'platform', 'repo_name', 'company'] + [col for col in df.columns if '2023-12' <= col <= '2024-11']
df_filtered = df[columns_to_keep]



fields_to_keep = ['Huawei', 'Alibaba', 'openKylin']
df_filtered = df_filtered[df_filtered['company'].isin(fields_to_keep)]


# 重置索引，避免 'field' 成为索引
df_filtered = df_filtered.reset_index(drop=True)


# 处理缺失数据
def handle_missing_data(group):
    # 将 0 视为缺失值，将其替换为 NaN
    group.loc[:, '2023-12':'2024-11'] = group.loc[:, '2023-12':'2024-11'].replace(0, np.nan)
    # 计算 2023-12 至 2024-11 期间缺失数据的个数
    missing_count = group.loc[:, '2023-12':'2024-11'].isna().sum(axis=1)
    # 若缺失数据个数 <= 3，则插值处理，否则删除该条数据
    group = group[missing_count <= 3]
    # 插值处理：线性插值
    group.loc[:, '2023-12':'2024-11'] = group.loc[:, '2023-12':'2024-11'].interpolate(method='linear', limit_direction='both')
    return group



df_filtered = df_filtered.groupby('company', group_keys=False).apply(handle_missing_data)


# 计算每个分组的平均值，保留两位小数
def mean_2023_2024(group):
    return group.loc[:, '2023-12':'2024-11'].mean().round(2)


df_avg = df_filtered.groupby('company', group_keys=False).apply(mean_2023_2024)


# 计算 2023-12 至 2024-11 所有列的总活跃度
df_filtered['total_activity'] = df_filtered.loc[:, '2023-12':'2024-11'].sum(axis=1)


# 根据总活跃度排序，选择前 5 名
df_top_5 = df_filtered.groupby('company', group_keys=False).apply(lambda x: x.nlargest(5, 'total_activity'))


# 输出结果
print("分组平均值：")
print(df_avg)
print("\n每组 top 5：")
print(df_top_5)

分组平均值：
           2023-12  2024-01  2024-02  2024-03  2024-04  2024-05  2024-06  \
company                                                                    
Alibaba       8.61     5.33     5.99    12.05    12.06     7.24     5.10   
Huawei        5.85     5.04     4.56     6.40     6.37     5.83     5.73   
openKylin    24.58    20.36    16.50    25.34    29.31    25.39    24.14   

           2024-07  2024-08  2024-09  2024-10  2024-11  
company                                                 
Alibaba       5.90     7.05     4.73     4.22     3.57  
Huawei        5.24     5.69     5.79     4.71     4.62  
openKylin    26.76    28.47    28.53    24.71    27.46  

每组 top 5：
             id platform                             repo_name    company  \
13619  23271574    gitee           anolis-challenge/summer2022    Alibaba   
13663  23397622    gitee                 src-openeuler/buildah    Alibaba   
15728   6959873    gitee                 src-openeuler/libtiff    Alibaba   
15154   

In [2]:

df_avg

,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
company,,,,,,,,,,,,
Alibaba,8.61,5.33,5.99,12.05,12.06,7.24,5.10,5.90,7.05,4.73,4.22,3.57
Huawei,5.85,5.04,4.56,6.40,6.37,5.83,5.73,5.24,5.69,5.79,4.71,4.62
openKylin,24.58,20.36,16.50,25.34,29.31,25.39,24.14,26.76,28.47,28.53,24.71,27.46


In [3]:
df_top_5

,id,platform,repo_name,company,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,total_activity
13619,23271574,gitee,anolis-challenge/summer2022,Alibaba,9.500,8.55,15.10,42.03,48.18,13.230,12.95,14.44,12.520,9.720000,11.750,16.50,214.470000
13663,23397622,gitee,src-openeuler/buildah,Alibaba,13.375,8.61,11.30,11.24,16.14,7.625,4.25,1.95,3.380,6.963333,3.970,1.81,90.613333
15728,6959873,gitee,src-openeuler/libtiff,Alibaba,16.490,5.31,0.72,2.77,3.50,12.240,2.90,14.70,20.640,6.070000,2.758,1.43,89.528000
15154,9932270,gitee,src-openeuler/python-jwcrypto,Alibaba,17.250,4.60,0.72,15.61,4.98,2.020,1.43,2.02,6.800,4.206667,3.566,2.05,65.252667
17121,11572642,gitee,src-openeuler/git-lfs,Alibaba,2.220,1.78,7.64,4.64,6.67,4.230,2.97,2.67,2.020,3.970000,1.950,0.72,41.480000
4036,13516456,gitee,src-openeuler/chromium,Huawei,13.010,19.81,14.34,29.80,9.32,9.350,10.31,12.41,13.220,37.910000,13.070,7.69,190.240000
1484,15736142,gitee,src-openeuler/ffmpeg,Huawei,4.690,4.02,11.84,2.02,18.89,5.590,44.03,43.70,23.220,16.040000,4.400,11.13,189.570000
1407,7117195,gitee,openeuler/openeuler-jenkins,Huawei,13.050,19.76,9.16,11.24,18.82,7.410,5.96,19.93,26.330,9.880000,6.450,12.23,160.220000
1153,7428316,gitee,src-openeuler/wireshark,Huawei,2.740,16.73,6.32,14.54,3.90,18.010,4.30,3.19,18.970,9.050000,26.620,12.94,137.310000
1531,12758843,gitee,mindspore/serving,Huawei,20.110,29.55,11.73,21.89,2.47,9.110,2.68,4.75,5.370,6.260000,12.720,9.57,136.210000
